<a href="https://colab.research.google.com/github/Taichi2005/GoogleColab_Whisper/blob/main/%E3%80%90%E6%A9%9F%E8%83%BD%E6%8B%A1%E5%BC%B5%E7%89%88%E3%80%91%E6%9C%80%E7%B5%82%E7%A2%BA%E5%AE%9A%E3%82%B3%E3%83%BC%E3%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# セル1: 環境構築
# -------------------------------------------------------------------------------------------
# GPUが有効になっているかを確認
print("▼ GPUの確認")
!nvidia-smi

# faster-whisperと、GPUで動作させるために必要なCUDAライブラリをインストール
print("\n▼ 必要なライブラリのインストール")
!pip install --upgrade faster-whisper -q
!pip install nvidia-cublas-cu12==12.6.4.1 -q
!pip install nvidia-cudnn-cu12==9.10.2.21 -q

print("\n✅ 環境構築が完了しました。")

In [ ]:
# -------------------------------------------------------------------------------------------
# セル2: Google Driveへの接続
# -------------------------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')
print("\n✅ Google Driveへの接続が完了しました。")

In [ ]:
#@title 🚀【機能拡張版】高性能・多機能 文字起こし実行セル
#@markdown ### 1. Google Driveのパス設定
#@markdown ---
#@markdown 音声や動画ファイルが保存されているフォルダと、結果を保存するフォルダのパスを指定します。
drive_audio_input_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/input_audio" #@param {type:"string"}
drive_transcript_output_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts" #@param {type:"string"}

#@markdown ### 2. 処理対象の選択
#@markdown ---
#@markdown 入力フォルダ内のファイルをどのように処理するか選択してください。
processing_mode = "\u5168\u3066\u306E\u30D5\u30A1\u30A4\u30EB\u3092\u51E6\u7406" #@param ["全てのファイルを処理", "特定の１ファイルのみ処理"]
#@markdown ---
#@markdown `特定の１ファイルのみ処理` を選択した場合は、下のフィールドに対象のファイル名を入力してください。
specific_filename = "your_audio_file.mp4" #@param {type:"string"}


#@markdown ### 3. モデルとパフォーマンス設定
#@markdown ---
#@markdown **モデル**: `Zoont/faster-whisper-large-v3-turbo-int8-ct2`は、精度を維持しつつメモリ効率を最大化した推奨モデルです。<br>
#@markdown **計算タイプ**: `int8_float16`は、int8モデル利用時の推奨設定です。
model_name = "Zoont/faster-whisper-large-v3-turbo-int8-ct2" #@param ["Zoont/faster-whisper-large-v3-turbo-int8-ct2", "deepdml/faster-whisper-large-v3-turbo-ct2", "large-v3"]
compute_type = "int8_float16" #@param ["int8_float16", "float16", "int8"]


#@markdown ### 4. VAD (音声区間検出) 設定
#@markdown ---
#@markdown VADを有効にすると、無音区間を自動で除去し、文字起こしの精度を向上させることができます。
use_vad_filter = True #@param {type:"boolean"}
#@markdown ここで指定したミリ秒以上の無音を「発話の区切り」とみなします。
vad_min_silence_duration_ms = 500 #@param {type:"slider", min:100, max:2000, step:100}

# --- ここから下は実行コード（編集不要） ---
import os
import glob
import shutil
from faster_whisper import WhisperModel
import time
from datetime import datetime
from tqdm.notebook import tqdm

def get_timestamp():
    """現在時刻をフォーマットして返す"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def save_transcription_result(segments_generator, output_path):
    """文字起こし結果(ジェネレータ)をファイルに書き出す関数"""
    with open(output_path, "w", encoding="utf-8") as f:
        for segment in segments_generator:
            line = f"[{segment.start:0>7.2f}s -> {segment.end:0>7.2f}s] {segment.text.strip()}\n"
            f.write(line)

# 1. 環境設定とディレクトリの準備
print(f"[{get_timestamp()}] --- 1. 環境設定とディレクトリの準備 ---")
os.makedirs(drive_audio_input_dir, exist_ok=True)
os.makedirs(drive_transcript_output_dir, exist_ok=True)
print(f"入力フォルダ: {drive_audio_input_dir}")
print(f"出力フォルダ: {drive_transcript_output_dir}")

# 2. モデルのロード
print(f"\n[{get_timestamp()}] --- 2. モデル '{model_name}' ({compute_type}) をロード中... ---")
start_load_time = time.time()
model = WhisperModel(model_name, device="cuda", compute_type=compute_type)
end_load_time = time.time()
print(f"✅ モデルのロードが完了しました。({end_load_time - start_load_time:.2f}秒)")

# 3. 処理対象ファイルの検索と選択
print(f"\n[{get_timestamp()}] --- 3. 処理対象ファイルの選択 ---")
supported_audio_formats = ['*.mp3', '*.wav', '*.m4a', '*.flac', '*.ogg', '*.opus']
supported_video_formats = ['*.mp4', '*.mov', '*.avi', '*.wmv']
all_media_files = []
for fmt in supported_audio_formats + supported_video_formats:
    all_media_files.extend(glob.glob(os.path.join(drive_audio_input_dir, fmt)))

files_to_process = []
if processing_mode == "特定の１ファイルのみ処理":
    target_path = os.path.join(drive_audio_input_dir, specific_filename)
    if target_path in all_media_files:
        files_to_process.append(target_path)
        print(f"✅ 特定ファイルモード: '{specific_filename}' を処理対象とします。")
    else:
        print(f"⚠️ エラー: 指定されたファイル '{specific_filename}' が入力フォルダ内に見つかりません。")
else:
    files_to_process = all_media_files
    print(f"✅ {len(files_to_process)} 件の全ファイルを処理対象とします。")

# 4. メイン処理
if not files_to_process:
    print("\n--- 処理対象ファイルがないため、終了します。 ---")
else:
    print(f"\n[{get_timestamp()}] --- 4. 文字起こし処理開始 ---")
    overall_start_time = time.time()

    # tqdmを使用してプログレスバーを表示
    for audio_path_drive in tqdm(files_to_process, desc="Overall Progress"):
        start_process_time = time.time()
        filename = os.path.basename(audio_path_drive)
        base_filename, file_extension = os.path.splitext(filename)
        print(f"\n[{get_timestamp()}] 処理開始: {filename}")

        local_media_path = None
        temp_audio_file = None
        path_to_transcribe = None

        try:
            # 元ファイルをローカルにコピー
            local_media_path = os.path.join('/content/', filename)
            shutil.copy(audio_path_drive, local_media_path)
            print(f"  - ファイルをローカルにコピーしました。")

            # 動画ファイルかどうかを判定
            if file_extension.lower() in [ext.strip('*') for ext in supported_video_formats]:
                print(f"  - 動画ファイルを検出。音声を抽出します...")
                # 抽出後の音声ファイルパスを定義
                temp_audio_file = os.path.join('/content/', f"{base_filename}.mp3")
                # FFmpegコマンドを構築（-y:上書き, -i:入力, -vn:ビデオ無効, -ar:サンプルレート, -ac:チャンネル数, -loglevel:ログレベル）
                ffmpeg_command = f'ffmpeg -y -i "{local_media_path}" -vn -ar 16000 -ac 1 -loglevel error "{temp_audio_file}"'
                # コマンド実行
                os.system(ffmpeg_command)
                path_to_transcribe = temp_audio_file
                print(f"  - 音声の抽出が完了 -> {os.path.basename(temp_audio_file)}")
            else:
                # 音声ファイルの場合はそのまま処理
                path_to_transcribe = local_media_path

            # 文字起こし実行
            print(f"  - 文字起こしを実行中...")
            segments, info = model.transcribe(
                path_to_transcribe,
                beam_size=5,
                vad_filter=use_vad_filter,
                vad_parameters=dict(min_silence_duration_ms=vad_min_silence_duration_ms)
            )
            print(f"  - 検出言語: {info.language} (確率: {info.language_probability:.2f})")

            # 結果を保存
            output_txt_filename = f"{base_filename}.txt"
            local_output_path = os.path.join('/content/', output_txt_filename)
            drive_output_path = os.path.join(drive_transcript_output_dir, output_txt_filename)

            save_transcription_result(segments, local_output_path)

            shutil.move(local_output_path, drive_output_path)
            print(f"  - 結果を保存しました: {drive_output_path}")

        except Exception as e:
            print(f"  - 💥 エラーが発生しました: {e}")
        finally:
            # 一時ファイルをクリーンアップ
            if local_media_path and os.path.exists(local_media_path):
                os.remove(local_media_path)
            if temp_audio_file and os.path.exists(temp_audio_file):
                os.remove(temp_audio_file)
            end_process_time = time.time()
            print(f"  - 処理完了 ({end_process_time - start_process_time:.2f}秒)")

    overall_end_time = time.time()
    print(f"\n[{get_timestamp()}] --- 🎉 全てのファイルの処理が完了しました (合計所要時間: {overall_end_time - overall_start_time:.2f}秒) ---")